In [74]:
import os
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models, regularizers, callbacks
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import load_model
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
from tensorflow.keras.optimizers.schedules import ExponentialDecay

In [48]:
# Configuración inicial
train_dir = 'model_evaluation/train'
test_dir = 'model_evaluation/test'
img_size = (224, 224)
batch_size = 32

In [52]:
# Crear generadores de datos con augmentación moderada
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,  # Mantener aumentación moderada
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=[0.9, 1.1],
    horizontal_flip=True,
    fill_mode='nearest'
)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='binary'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='binary',
    shuffle=False
)

Found 15132 images belonging to 2 classes.
Found 3932 images belonging to 2 classes.


In [53]:
# Cargar el modelo preentrenado
model = load_model('models/best_model_fold_2.keras')

In [81]:
# Modificar la arquitectura del modelo para fine-tuning
inputs = model.input
x = model.layers[-2].output

# Añadir Dropout si no está ya en el modelo
if not any(isinstance(layer, layers.Dropout) for layer in model.layers):
    x = layers.Dropout(0.3)(x)  # Ajustar el Dropout a un 30%

# Añadir la nueva capa Dense con regularización L2
x = layers.Dense(1, activation='sigmoid', kernel_regularizer=regularizers.l2(0.001))(x)

# Crear el nuevo modelo
model = models.Model(inputs=inputs, outputs=x)

# Configurar el optimizador con un learning rate fijo
initial_learning_rate = 5e-5  # Ajustar según sea necesario
optimizer = Adam(learning_rate=initial_learning_rate)  # Fijo, no LearningRateSchedule

# Compilar el modelo con pérdida binaria y el nuevo optimizador
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])


In [83]:
# Obtener el índice de las clases desde el generador
melanoma_index = train_generator.class_indices['Melanoma']
not_melanoma_index = train_generator.class_indices['NotMelanoma']

# Contar el número de imágenes en cada clase
melanoma_count = np.sum(train_generator.classes == melanoma_index)
not_melanoma_count = np.sum(train_generator.classes == not_melanoma_index)
total_images = melanoma_count + not_melanoma_count

# Calcular los pesos de las clases
class_weights = {
    melanoma_index: total_images / (2 * melanoma_count),  # Peso para Melanoma
    not_melanoma_index: total_images / (2 * not_melanoma_count)  # Peso para NotMelanoma
}

print(f"Pesos de las clases calculados: {class_weights}")

Pesos de las clases calculados: {0: 2.0465242088179605, 1: 0.6616528202885876}


In [105]:
# Verificar un batch del generador de entrenamiento
images, labels = next(train_generator)
print(f"Tamaño de las imágenes: {images.shape}")
print(f"Tamaño de las etiquetas: {labels.shape}")

Tamaño de las imágenes: (32, 224, 224, 3)
Tamaño de las etiquetas: (32,)


In [108]:
# Configurar los callbacks
checkpoint = callbacks.ModelCheckpoint("fine_tuned_model.weights.h5", monitor='val_loss', save_best_only=True, mode='min', save_weights_only=True)
early_stopping = callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
reduce_lr = callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=1e-6)

# Callback adicional para respaldos de pesos
class BackupCheckpoint(callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        # Validar logs y evitar errores si es None
        if logs is None:
            logs = {}
        # Guardar los pesos del modelo como backup
        self.model.save_weights(f"backup_epoch_{epoch+1}.weights.h5")

backup_checkpoint = BackupCheckpoint()

In [109]:
# Entrenar el modelo
history = model.fit(
    train_generator,
    validation_data=test_generator,
    epochs=70,
    initial_epoch=48,  # Ajusta este valor al último epoch completado
    class_weight=class_weights,
    callbacks=[checkpoint, early_stopping, reduce_lr, backup_checkpoint]
)

Epoch 49/70


AttributeError: 'NoneType' object has no attribute 'items'

In [106]:
# Guardar el modelo después del entrenamiento
model.save('fine_tuned_model_final.keras')

# Evaluar el modelo en el conjunto de prueba

In [107]:
# Evaluar el modelo en el conjunto de prueba
evaluation = model.evaluate(test_generator)
print(f'Test Loss: {evaluation[0]:.4f}')
print(f'Test Accuracy: {evaluation[1]*100:.2f}%')

# Obtener predicciones para cada imagen en el conjunto de prueba
predictions = model.predict(test_generator, verbose=1)
predicted_classes = (predictions > 0.5).astype(int).flatten()
true_classes = test_generator.classes

123/123 ━━━━━━━━━━━━━━━━━━━━ 527s 4s/step - accuracy: 0.4612 - loss: 0.7768
Test Loss: 0.7556
Test Accuracy: 75.05%
123/123 ━━━━━━━━━━━━━━━━━━━━ 490s 4s/step


In [ ]:
# Generar el informe de clasificación
class_labels = list(test_generator.class_indices.keys())
report = classification_report(true_classes, predicted_classes, target_names=class_labels)
print("\nClassification Report:")
print(report)

# Mostrar la matriz de confusión
conf_matrix = confusion_matrix(true_classes, predicted_classes)
print("\nConfusion Matrix:")
print(conf_matrix)

In [ ]:
# Visualización de la matriz de confusión
plt.figure(figsize=(8, 6))
plt.imshow(conf_matrix, cmap='Blues')
plt.title('Confusion Matrix')
plt.colorbar()
tick_marks = np.arange(len(class_labels))
plt.xticks(tick_marks, class_labels, rotation=45)
plt.yticks(tick_marks, class_labels)
plt.ylabel('True Label')
plt.xlabel('Predicted Label')

# Añadir los números en la matriz de confusión
for i in range(len(class_labels)):
    for j in range(len(class_labels)):
        plt.text(j, i, str(conf_matrix[i, j]), ha='center', va='center', color='red')

plt.tight_layout()
plt.show()

# EXPLICACIONES

Explicación:
Carga del Modelo Preentrenado: El modelo se carga desde models/best_model_fold_2.keras.

Ajuste del Modelo:

Se verifica la última capa del modelo y se ajusta para clasificación binaria si es necesario.
Se añade Dropout después de las capas densas si no estaba presente.
Compilación y Entrenamiento: El modelo se recompila y se entrena con los datos nuevos de train_dir y test_dir.

Respaldo: Se incluye un callback personalizado para guardar los pesos después de cada época (backup_checkpoint).

Resumen:
Este código te permitirá hacer fine-tuning sobre el modelo que ya entrenaste, utilizando las nuevas imágenes y ajustando el modelo para manejar el desbalanceo de clases. Además, asegura que no perderás el progreso si el entrenamiento se interrumpe.

La estrategia propuesta es sólida y está bien encaminada, pero podemos considerar algunos puntos para maximizar su efectividad. Aquí hay algunas consideraciones que podrías evaluar para asegurarte de que la estrategia sea la mejor:

1. Carga del Modelo Preentrenado y Fine-Tuning:
Pros: Aprovechas un modelo ya entrenado y bien ajustado, lo que debería acelerar el proceso de convergencia y mejorar la precisión en el nuevo conjunto de datos.
Cons: Si el modelo original estaba sobreajustado (overfitted) al conjunto de datos original, podría tener problemas para generalizar en los nuevos datos, incluso con el fine-tuning. En este caso, ajustar la estructura del modelo (añadir regularización o Dropout) es esencial.
2. Manejo del Desbalance de Clases:
Pros: Usar ponderación de clases es una estrategia común y efectiva para manejar el desbalance en los datos, especialmente en problemas binarios como este.
Cons: Aunque ponderar las clases ayuda, puede ser complementado con técnicas de aumentación de datos para la clase minoritaria, o incluso con técnicas más avanzadas como Oversampling (SMOTE) en el espacio de los datos de entrenamiento.
3. Regularización y Dropout:
Pros: Agregar regularización y Dropout es una buena estrategia para prevenir el sobreajuste, especialmente cuando el nuevo conjunto de datos es más grande y diverso.
Cons: La cantidad de Dropout debe ser ajustada cuidadosamente; demasiado Dropout puede reducir la capacidad de aprendizaje del modelo, mientras que demasiado poco puede no ser suficiente para prevenir el sobreajuste.
4. Verificación de Progreso y Backup de Pesos:
Pros: Implementar un backup regular de los pesos garantiza que no se pierda el progreso en caso de interrupciones.
Cons: Asegúrate de monitorizar los pesos guardados para evitar guardar un modelo que se esté sobreajustando, lo que podría suceder si el proceso se reanuda muchas veces.
5. Monitoreo de Métricas de Validación:
Pros: El uso de callbacks como EarlyStopping y ReduceLROnPlateau es excelente para ajustar dinámicamente el aprendizaje y evitar sobreajuste.
Cons: Es vital monitorizar no solo la pérdida (loss) sino también la precisión (accuracy) y otras métricas como la f1-score, especialmente en casos de desbalance de clases.
6. Ajuste Final del Modelo:
Pros: Cargar el mejor modelo basado en la validación cruzada y luego hacer fine-tuning con datos adicionales puede consolidar el rendimiento.
Cons: Después del fine-tuning, es crucial validar nuevamente el modelo con un conjunto de datos completamente nuevo o realizar una validación cruzada adicional.
Conclusión:
Esta estrategia es sólida, pero tiene algunos aspectos que pueden ser optimizados o ajustados según los resultados que vayas observando en cada paso. La clave es asegurarse de que el modelo no se sobreajuste y de que las métricas reflejen una buena generalización a datos no vistos. Además, mantener un enfoque iterativo, ajustando parámetros según sea necesario, te permitirá obtener los mejores resultados posibles.